In [17]:
import os
import json
import pandas as pd
from tqdm import tqdm
from tqdm.contrib.concurrent import thread_map
from paddlenlp import Taskflow

In [23]:
model_dir = r"G:\code\github\PaddleNLP\outputs\tnews_public\ernie-3.0-xbase-zh\export"

cls = Taskflow("text_classification", task_path=model_dir, is_static_model=True)

[2023-08-23 21:29:22,156] [    INFO] - We are using <class 'paddlenlp.transformers.ernie.tokenizer.ErnieTokenizer'> to load 'G:\code\github\PaddleNLP\outputs\tnews_public\ernie-3.0-xbase-zh\export'.
[2023-08-23 21:29:29,205] [    INFO] - Load id2label from G:\code\github\PaddleNLP\outputs\tnews_public\ernie-3.0-xbase-zh\export\id2label.json.


In [24]:
cls(["景甜出演我的女孩"])

[{'predictions': [{'label': 'news_entertainment',
    'score': 0.9744371611866152}],
  'text': '景甜出演我的女孩'}]

需要输出成这样的格式

```json
{"id": 0, "label": "102", "label_desc": "news_entertainment"}
{"id": 1, "label": "116", "label_desc": "news_game"}
{"id": 2, "label": "107", "label_desc": "news_car"}
```

In [25]:
data_dir = r"G:\dataset\text_classify\tnews_public\raw"

label_file = os.path.join(data_dir, "labels.json")
label2id = dict()
with open(label_file, "r", encoding="utf-8") as f:
    for line in f:
        line = json.loads(line)
        label_id = line["label"]
        label = line["label_desc"]
        label2id[label] = label_id

print(label2id)

{'news_story': '100', 'news_culture': '101', 'news_entertainment': '102', 'news_sports': '103', 'news_finance': '104', 'news_house': '106', 'news_car': '107', 'news_edu': '108', 'news_tech': '109', 'news_military': '110', 'news_travel': '112', 'news_world': '113', 'news_stock': '114', 'news_agriculture': '115', 'news_game': '116'}


In [26]:
def predict_one_line(line: str) -> str:
    """
    预测一行数据
    """
    line = json.loads(line)
    id = line["id"]
    sentence = line["sentence"]
    
    result = cls([sentence])
    label = result[0]["predictions"][0]["label"]
    label_id = label2id[label]

    return json.dumps({"id": id, "label": label_id, "label_desc": label}, ensure_ascii=False)

In [27]:
def predict_file(input_file: str, output_file: str):
    with open(input_file, "r", encoding="utf-8") as f:
        lines = []
        for line in tqdm(f):
            lines.append(predict_one_line(line))
    with open(output_file, "w", encoding="utf-8") as f:
        f.write("\n".join(lines))

In [28]:
output_dir = r"G:\dataset\text_classify\tnews_public\submit"

file = os.path.join(data_dir, "test1.0.json")
output_file = os.path.join(output_dir, "tnews10_predict.json")
predict_file(file, output_file)

file = os.path.join(data_dir, "test.json")
output_file = os.path.join(output_dir, "tnews11_predict.json")
predict_file(file, output_file)

10000it [00:40, 245.99it/s]
10000it [00:40, 249.66it/s]
